<a href="https://colab.research.google.com/github/Nirav2218/FirstRepo/blob/main/native_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torch
! pip install transformers

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,AdamW,BertForSequenceClassification,TrainingArguments,Trainer
import torch
model_name="bert-base-uncased"

In [3]:
df = pd.read_csv("BankReviews.csv", encoding='latin1',usecols=["Stars","Reviews"])
df['Stars'] = df['Stars'].replace(1, 0)
df['Stars'] = df['Stars'].replace(5, 1)
df.head()

,Stars,Reviews
0,1,"Great job, Wyndham Capital! Each person was pr..."
1,1,Matthew Richardson is professional and helpful...
2,1,We had a past experience with Wyndham Mortgage...
3,1,We have been dealing with Brad Thomka from the...
4,1,I can't express how grateful I am for the supp...


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# Tokenize the training and testing data
encode_data = tokenizer(df["Reviews"].tolist(), padding=True, truncation=True, return_tensors="pt")
encode_data

{'input_ids': tensor([[  101,  2307,  3105,  ...,     0,     0,     0],
        [  101,  5487,  9482,  ...,     0,     0,     0],
        [  101,  2057,  2018,  ...,     0,     0,     0],
        ...,
        [  101,  2054,  1037,  ...,     0,     0,     0],
        [  101, 16360,  2001,  ...,     0,     0,     0],
        [  101,  1045,  2001,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [6]:
from torch.utils.data import  DataLoader,TensorDataset

encode_data['labels'] = torch.tensor(df["Stars"])
train_dataset = TensorDataset(encode_data['input_ids'], encode_data['attention_mask'], encode_data['labels'])
train_dataset

In [7]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
train_loader

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained(model_name)
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# Check the number of classes in your model
num_classes_model = model.config.num_labels
print("Number of classes in the model:", num_classes_model)


Number of classes in the model: 2


In [10]:
# Training loop
for epoch in range(num_epochs):

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [item.to(device) for item in batch]

        # input_ids=batch["input_ids"].to(device)
        # attention_mask=batch["attention_mask"].to(device)
        # labels=batch["labels"].to(device)

        outputs=model(input_ids, attention_mask=attention_mask,labels=labels)
        loss=outputs[0]
        loss.backward()
        optimizer.step()

model.eval()


# Save the trained model
torch.save(model.state_dict(), 'custom_bert_model')

In [23]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

model_params_path = 'custom_bert_model'

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.load_state_dict(torch.load(model_params_path, map_location='cpu'))

model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Replace with the tokenizer you used for fine-tuning

text = "it is disghusting no one is giving reply"

inputs = tokenizer(text, return_tensors="pt")

outputs = model(**inputs)

logits = outputs.logits

probabilities = torch.nn.functional.softmax(logits, dim=1)

predicted_label = torch.argmax(probabilities, dim=1).item()

print("Predicted Probabilities:", probabilities)
print("Predicted Label:", predicted_label)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Probabilities: tensor([[0.9905, 0.0095]], grad_fn=<SoftmaxBackward0>)
Predicted Label: 0
